In [2]:
import scrapy
import requests
from scrapy.http import HtmlResponse

In [4]:
URL = 'https://www.networkworld.com/cloud-computing/'
#response = HtmlResponse(url=URL)
html = requests.get( URL ).content
sel = scrapy.Selector( text = html )

In [16]:
sel.xpath('//a[contains(@class, "card")]/@href').extract()

['https://www.networkworld.com/article/1250120/startup-oxide-computing-seeks-to-put-the-cloud-back-in-on-prem-private-clouds.html',
 'https://www.networkworld.com/article/1251939/ibm-cloud-service-aims-to-deliver-secure-multicloud-connectivity.html',
 'https://www.networkworld.com/article/1251281/eu-approves-1-3b-in-aid-for-cloud-edge-computing.html',
 'https://www.networkworld.com/article/1250044/mainframe-modernization-gets-a-boost-from-kyndryl-aws-collaboration.html',
 'https://www.networkworld.com/article/1249451/cisco-aws-further-integrate-cloud-management-capabilities.html',
 'https://www.networkworld.com/article/1247908/alibaba-scraps-plans-to-hive-off-cloud-business-amid-growing-chip-uncertainties.html',
 'https://www.networkworld.com/article/1247268/microsofts-maia-ai-azure-cobalt-chips-to-rev-up-efficiency-performance.html',
 'https://www.networkworld.com/article/1247134/cloud-management-skills-gap-drives-hybrid-cloud-adoption.html',
 'https://www.networkworld.com/article/957

In [20]:
for a in sel.xpath('//a[contains(@class, "card")]'):
    print(a.xpath('//div[contains (@class, "card__info--light")]/span/text()').extract())

['Dec 12, 2023', '8 mins', 'Dec 07, 2023', '3 mins', 'Dec 05, 2023', '3 mins', 'Nov 30, 2023', '4 mins', 'Nov 28, 2023', '4 mins', 'Nov 17, 2023', '4 mins', 'Nov 15, 2023', '7 mins', 'Nov 14, 2023', '3 mins', 'Oct 25, 2023', '5 mins', 'Aug 22, 2023 ', '4 mins', 'Aug 22, 2023 ', '4 mins', 'Aug 09, 2023 ', '2 mins', 'Aug 01, 2023 ', '6 mins', 'Jul 27, 2023 ', '3 mins', 'Jul 18, 2023 ', '4 mins', 'Jul 14, 2023 ', '3 mins', 'Jun 21, 2023 ', '4 mins', 'Jun 15, 2023 ', '5 mins', 'Jun 14, 2023 ', '2 mins', 'Jun 08, 2023 ', '4 mins', 'Jun 06, 2023 ', '7 mins', 'Feb 15, 2024', '18:30-21:30 GMT', 'The Greenhouses, London', 'Mar 21, 2024', '9:30 am – 18:30', 'London, United Kingdom', 'May 16, 2024', '9:30 am – 18:30', 'London, United Kingdom', '15 Dec 2023', '15 Dec 2023', '15 Dec 2023', 'Oct 05, 2021 ', '25 mins', 'Jan 22, 2021 ', '34 mins', 'Jan 15, 2021 ', '26 mins', 'May 01, 2020 ', '3 mins', 'Dec 15, 2023', '4 mins', 'Dec 15, 2023', '6 mins', 'Dec 14, 2023', '7 mins', 'Sep 24, 2021', '20 min

In [ ]:
sel.xpath('//a[contains(@class, "card")]/@href').extract()